In [1]:
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(precision=3,suppress=True,linewidth=120)


/home/marcello/anaconda3/lib/python3.9/site-packages/numpy/_core/getlimits.py:545: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [2]:
Nm = 1024
max_nm = 128
eps = 1.e-8

for _ in range(Nm):
    n, m = np.random.choice( max_nm, 2)
    A = np.random.random((n,m))*2-1
    if m > 0 and n > 0 and 0.1 > np.random.random():
        A[:,0] = 0.
    
    # Compute the pseudo inverse using svd
    U,Dd,Vt = np.linalg.svd(A)
    D  = np.zeros((n,m))
    Di = np.zeros((m,n))
    for i in range(min(m,n)):
        D [i,i] = Dd[i]
        Di[i,i] = 1. / Dd[i] if np.abs(Dd[i]) > eps else 0.
    Ap = Vt.T @ Di @ U.T

    # Check it is correct
    try:
        # Check Ap is actually the pseudo inverse
        assert( np.allclose(A , A @ Ap @ A ))
        assert( np.allclose(Ap, Ap @ A @ Ap) )
        # Chech AAp and ApA are projectors
        assert( np.allclose( A@Ap , A@Ap @ A@Ap))
        assert( np.allclose( Ap@A , Ap@A @ Ap@A))
        # Check that Ap solves the least square problem
        b = np.random.random(n)
        x,_,_,_ = np.linalg.lstsq(A,b)
        assert(np.allclose(x, Ap@b))
    except AssertionError:
        print( 'Failed', n, m)
        np.set_printoptions(threshold=np.inf)
        print(A)
        print()
        print(Ap)
        print()
        raise AssertionError
print('Passed')

Passed


In [14]:
# UP decomposition
n = 8
A = np.random.random((n,n))*2-1
if m > 0 and n > 0 and 0.1 > np.random.random():
    A[:,0] = 0.

# Compute the polar using svd
W,D,Vt = np.linalg.svd(A)
U = W @ Vt
P = Vt.T @ np.diag(D) @ Vt

assert(np.allclose(A, U@P))
print('Passed')

print(A)
print(P)
print(np.diag(D))



Passed
[[-0.062  0.505 -0.115 -0.793 -0.447  0.955  0.878 -0.124]
 [ 0.458  0.254  0.035  0.749 -0.983  0.22  -0.021  0.993]
 [-0.    -0.746  0.606  0.194 -0.23  -0.431  0.209  0.203]
 [ 0.582  0.988 -0.035 -0.701  0.995  0.135 -0.92  -0.987]
 [-0.405  0.427  0.582  0.188  0.33  -0.776 -0.152  0.813]
 [ 0.784 -0.407  0.514 -0.722 -0.42  -0.851 -0.525 -0.476]
 [-0.824 -0.324 -0.281  0.658  0.022 -0.605 -0.487  0.595]
 [ 0.66  -0.352  0.962 -0.63  -0.176  0.749  0.248  0.817]]
[[ 1.328  0.063  0.438 -0.448 -0.276  0.277 -0.261 -0.257]
 [ 0.063  1.442 -0.27  -0.173  0.33   0.3   -0.148 -0.146]
 [ 0.438 -0.27   1.145 -0.377  0.007 -0.213  0.063  0.471]
 [-0.448 -0.173 -0.377  1.489 -0.169 -0.339 -0.094  0.603]
 [-0.276  0.33   0.007 -0.169  1.382 -0.085 -0.423 -0.466]
 [ 0.277  0.3   -0.213 -0.339 -0.085  1.654  0.589  0.051]
 [-0.261 -0.148  0.063 -0.094 -0.423  0.589  1.262  0.191]
 [-0.257 -0.146  0.471  0.603 -0.466  0.051  0.191  1.732]]
[[2.736 0.    0.    0.    0.    0.    0.    0. 

In [77]:
# Log of matrix
# we use P since it is >= 0

av = np.linalg.eigvals(A)
uv = np.linalg.eigvals(U)
pv = np.linalg.eigvals(P)

assert( np.isclose( np.prod(av) , np.prod(uv) * np.prod(pv) ) )


pu,pv = np.linalg.eigh(P)
vp = np.linalg.inv(pv)
tmp = vp @ P @ pv
print('tmp')
for i in range(n):
    if ( tmp[i,i] > 0 ):
        tmp[i,i] = np.log(tmp[i,i])
log_p = pv @ tmp @ vp

exp_log_p = np.zeros_like(P)
pn = np.eye(n)
fac_i = 1
for i in range(100):
    exp_log_p += pn / fac_i
    tmp = np.eye(n)
    fac_i *= i+1
    pn @= log_p

print(P)
print(log_p)
print(exp_log_p)
assert( np.allclose( P , exp_log_p) )

tmp
[[ 1.328  0.063  0.438 -0.448 -0.276  0.277 -0.261 -0.257]
 [ 0.063  1.442 -0.27  -0.173  0.33   0.3   -0.148 -0.146]
 [ 0.438 -0.27   1.145 -0.377  0.007 -0.213  0.063  0.471]
 [-0.448 -0.173 -0.377  1.489 -0.169 -0.339 -0.094  0.603]
 [-0.276  0.33   0.007 -0.169  1.382 -0.085 -0.423 -0.466]
 [ 0.277  0.3   -0.213 -0.339 -0.085  1.654  0.589  0.051]
 [-0.261 -0.148  0.063 -0.094 -0.423  0.589  1.262  0.191]
 [-0.257 -0.146  0.471  0.603 -0.466  0.051  0.191  1.732]]
[[-0.067  0.085  0.587 -0.203 -0.428  0.353 -0.434 -0.337]
 [ 0.085  0.275 -0.267 -0.132  0.255  0.184 -0.102  0.025]
 [ 0.587 -0.267 -0.341 -0.474  0.257 -0.366  0.226  0.632]
 [-0.203 -0.132 -0.474  0.135 -0.056 -0.267 -0.068  0.512]
 [-0.428  0.255  0.257 -0.056  0.079  0.084 -0.434 -0.406]
 [ 0.353  0.184 -0.366 -0.267  0.084  0.266  0.548  0.189]
 [-0.434 -0.102  0.226 -0.068 -0.434  0.548 -0.061 -0.016]
 [-0.337  0.025  0.632  0.512 -0.406  0.189 -0.016  0.231]]
[[ 1.328  0.063  0.438 -0.448 -0.276  0.277 -0.261

In [44]:
tmp

array([[ 0.146,  0.   , -0.   ,  0.   ,  0.   , -0.   , -0.   , -0.   ],
       [ 0.   ,  0.541, -0.   , -0.   , -0.   , -0.   , -0.   , -0.   ],
       [-0.   ,  0.   ,  0.867,  0.   ,  0.   , -0.   ,  0.   , -0.   ],
       [ 0.   , -0.   , -0.   ,  1.266, -0.   , -0.   , -0.   , -0.   ],
       [ 0.   ,  0.   ,  0.   , -0.   ,  1.468,  0.   ,  0.   ,  0.   ],
       [-0.   , -0.   , -0.   , -0.   , -0.   ,  2.018, -0.   ,  0.   ],
       [-0.   , -0.   , -0.   , -0.   ,  0.   ,  0.   ,  2.391,  0.   ],
       [-0.   , -0.   ,  0.   , -0.   , -0.   ,  0.   ,  0.   ,  2.736]])